# Parameter drift

# Purpose
If the matematical model is not correct or too little data is available this may lead to paramter drift, so that the parameters in the matematical model changes depending on how the fitted data has been sampled.

# Methodology
* Sample data of forces from a higher order model
* Fit a lower order model to a random sample of this data

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models.regression import ForceRegression, results_summary_to_dataframe
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.visualization.regression import show_pred, show_pred_captive
from vessel_manoeuvring_models.visualization.plot import track_plot,captive_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
import vessel_manoeuvring_models.models.vmm_nonlinear_EOM  as vmm_simpler

from vessel_manoeuvring_models.models.vmm import ModelSimulator

from vessel_manoeuvring_models.data.wpcc import ship_parameters, df_parameters, ps, ship_parameters_prime, ps_ship, scale_factor


In [ ]:
#format the book
import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

## Parameters

In [ ]:
run_params = {
    'experiment' : 'VCT_linear',
    'model_test_id' : 22770,
    'model_test_dir_path' : os.path.abspath('../../../data/processed/kalman'),
    'model' : os.path.abspath('../../../models/model_VCT_abkowitz.pkl'),
    'run_name' : '22770',
}

In [ ]:
model = ModelSimulator.load('../models/model_VCT_abkowitz.pkl')

## Load VCT data

In [ ]:
df_VCT_all = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT = df_VCT_all.groupby(by=['model_name']).get_group('V2_5_MDL_modelScale')

# Subtract the resistance

In [ ]:
df_resistance = df_VCT.groupby(by='test type').get_group('resistance')

X = df_resistance[['u','fx']].copy()
X['u**2'] = X['u']**2
y = X.pop('fx')

model_resistance = sm.OLS(y,X)
results_resistance = model_resistance.fit()

X_pred = pd.DataFrame()
X_pred['u'] = np.linspace(X['u'].min(), X['u'].max(), 20)
X_pred['u**2'] = X_pred['u']**2
X_pred['fx'] = results_resistance.predict(X_pred)

fig,ax=plt.subplots()
df_resistance.plot(x='u', y='fx', style='.', ax=ax)
X_pred.plot(x='u', y='fx', style='--', ax=ax);


In [ ]:
df_VCT_0_resistance = df_VCT.copy()
df_VCT_0_resistance['u**2'] = df_VCT_0_resistance['u']**2
df_VCT_0_resistance['fx']-= results_resistance.predict(df_VCT_0_resistance[['u','u**2']])

## VCT to prime system

In [ ]:
interesting = [
    'u',
    'v',
    'r',
    'delta',
    'fx',
    'fy',
    'mz',
    'thrust',
]
#df_VCT_prime = ps_ship.prime(df_VCT[interesting], U=df_VCT['V'])
df_VCT_prime = ps_ship.prime(df_VCT_0_resistance[interesting], U=df_VCT_0_resistance['V'])

## Captive test

In [ ]:
outputs = model.forces(inputs = df_VCT_prime)
df_captive = pd.merge(left=outputs, right=df_VCT_prime, 
                      how='left', 
                      left_index=True, 
                      right_index=True,
                      suffixes = ('','_vct'),
                      )

df_captive['test type'] = df_VCT['test type']
captive_plot(df_captive=df_captive, suffixes=['_vct'], legends = ['complex model','VCT'])

In [ ]:
result = model.zigzag(u0=2.0, angle=30)

In [ ]:
result.track_plot()

In [ ]:
result.plot(compare=False);

In [ ]:
df_sim_result = result.result.copy()
df_sim_result = ps.prime(df_sim_result, U=df_sim_result['U'])

## Fit a lower order model to this captive dataset

### Regression

In [ ]:
reg = ForceRegression(vmm=vmm_simpler, data=df_captive)
display(reg.show_pred_X())
display(reg.show_pred_Y())
display(reg.show_pred_N())

In [ ]:
model_vct = reg.create_model(df_parameters=df_parameters, ship_parameters=ship_parameters, ps=ps)

In [ ]:
outputs = model_vct.forces(inputs = df_VCT_prime)
df_captive_all = pd.merge(left=df_captive, right=outputs, 
                      how='left', 
                      left_index=True, 
                      right_index=True,
                      suffixes = ('','_simple'),
                      )

captive_plot(df_captive=df_captive_all, suffixes=['_vct','_simple'], 
             legends = ['complex model', 'VCT'], styles=['-', '.', '--'])

In [ ]:
result2 = model_vct.simulate(df_=result.result)

In [ ]:
result2.plot_compare(True)

In [ ]:
df_captive = df_VCT_prime.copy()
df_captive['test type'] = df_VCT['test type']

N = len(df_captive)
N_sample = N - 5
df_captive_sample = df_captive.sample(n=N_sample, random_state=42)

In [ ]:
model_names = [f'{i}' for i in range(20)]

df_captive_all = df_captive.copy()
np.random.seed(42)

models = {}

for model_name in model_names:
    
    df_captive_sample = df_captive.sample(n=N_sample)
    
    reg = ForceRegression(vmm=vmm_simpler, data=df_captive_sample)
    model_reg = reg.create_model(df_parameters=df_parameters, ship_parameters=ship_parameters, ps=ps)
    
    models[model_name] = model_vct = reg.create_model(df_parameters=df_parameters, ship_parameters=ship_parameters, ps=ps)
    
    
    outputs = model_reg.forces(inputs = df_captive)
    df_captive_all = pd.merge(left=df_captive_all, right=outputs, 
                          how='left', 
                          left_index=True, 
                          right_index=True,
                          suffixes = ('',f'_{model_name}'),
                          )
    

suffixes = [f'_{model_name}' for model_name in model_names]
styles = ['.'] + ['-' for model_name in model_names]

legends = ['VCT'] + model_names

captive_plot(df_captive=df_captive_all, suffixes=suffixes, 
             legends = legends, styles=styles)

In [ ]:
df_results = pd.DataFrame()
for model_name, model in models.items():
    result_ = model.simulate(df_=result.result)
    df_ = result_.result
    df_['t'] = df_.index
    df_['model_name'] = model_name
    df_results = df_results.append(df_, ignore_index=True)
    

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot

fig,ax=plt.subplots()
fig.set_size_inches(15,15)

for model_name, df_ in df_results.groupby(by='model_name'):
    
    df_.plot(x='y0', y='x0', ax=ax, alpha=0.05, lw=10, style='b-')

#result.result.plot(x='y0', y='x0', ax=ax)
    
ax.set_xlabel('y0 [m]')   
ax.set_ylabel('x0 [m]')   

ax.set_aspect("equal")
ax.set_title("Track plot")
ax.get_legend().set_visible(False)
ax.grid(True)


In [ ]:
#fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
#
#df_results['psi_deg'] = np.rad2deg(df_['psi'])
#
#for model_name, df_ in df_results.groupby(by='model_name'):
#    
#    df_.plot(x='t', y='v', ax=ax)
#
#ax.set_xlabel('time [s]')   
#ax.set_ylabel('Heading $\psi$ [deg]')   
#
#ax.set_title("Time series")
#ax.get_legend().set_visible(False)
#ax.grid(True)
